In [1]:
import re
import time

import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pymongo import MongoClient
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.notebook import tqdm


# Configuration
### Select parameters for cleaning

In [2]:
ngram_length = 3
min_yearly_df = 5

# Cleaning pipeline

In [3]:
alphabets = "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
htmltags = '<[^>]+>'
htmlspecial = '&#?[xX]?[a-zA-Z0-9]{2,8};'

start_delimiter = 'documentstart'
sent_delimiter = 'sentenceboundary'
end_delimiter = 'documentend'

delimiters = [start_delimiter, sent_delimiter, end_delimiter]

# Download the lemmatisesr
wnl = WordNetLemmatizer()

# Create a tokeniser
count = CountVectorizer(strip_accents='ascii', min_df=1)
tokeniser = count.build_analyzer()


def normalise_acronymns(text):
    '''
    Remove the periods in acronyms. 
    Adapted from the method found at https://stackoverflow.com/a/40197005 
    '''
    return re.sub(r'(?<!\w)([A-Z, a-z])\.', r'\1', text)


def normalise_decimals(text):
    '''
    Remove the periods in decimal numbers and replace with POINT
    '''
    return re.sub(r'([0-9])\.([0-9])', r'\1POINT\2', text)


def split_into_sentences(text):
    '''
    Sentence splitter adapted from https://stackoverflow.com/a/31505798
    '''
    text = text.replace("\n", " ")
    text = re.sub(prefixes, "\\1<prd>", text)
    text = re.sub(websites, "<prd>\\1", text)

    # my addition
    text = re.sub(htmltags, " ", text)
    text = re.sub(htmlspecial, " ", text)

    if "Ph.D" in text:
        text = text.replace("Ph.D.", "PhD")

    text = re.sub("\s" + alphabets + "[.] ", " \\1", text)
    text = re.sub(acronyms+" "+starters, "\\1<stop> \\2", text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]", "\\1\\2\\3", text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]", "\\1\\2", text)
    text = re.sub(" "+suffixes+"[.] "+starters, " \\1 \\2", text)
    text = re.sub(" "+suffixes+"[.]", " \\1", text)
    text = re.sub(" " + alphabets + "[.]", " \\1", text)

    if "”" in text:
        text = text.replace(".”", "”.")
    if "\"" in text:
        text = text.replace(".\"", "\".")
    if "!" in text:
        text = text.replace("!\"", "\"!")
    if "?" in text:
        text = text.replace("?\"", "\"?")

    text = text.replace(".", "<stop>")
    text = text.replace("?", "<stop>")
    text = text.replace("!", "<stop>")

    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]

    non_empty = []

    for s in sentences:
        # we require that there be two alphanumeric characters in a row
        if len(re.findall("[A-Za-z0-9][A-Za-z0-9]", s)) > 0:
            non_empty.append(s)
    return non_empty


def pad_sentences(sentences):
    '''
    Takes a list of sentences and returns a string in which:
        - The beginning of the abstract is indicated by DOCUMENTSTART
        - The end is indicated by DOCUMENTEND
        - Sentence boundaries are indicated by SENTENCEBOUNDARY

    The number of delimiters used is dependent on the ngram length
    '''
    sent_string = (' '+(sent_delimiter+' ')*(ngram_length-1)).join(sentences)

    return (start_delimiter+' ')*(ngram_length-1) + sent_string + (' '+end_delimiter)*(ngram_length-1)

def get_stopwords():
    stop = set(stopwords.words('english'))
    stop = set([s.replace("'", "") for s in stop])

    # Add years to prevent spikes
    for year in range(1900, 2020):
        stop.add(str(year))

    # Add small numbers
    for num in range(0, 100):
        if len(str(num)) < 2:
            stop.add(str(num))
            num = '0' + str(num)
            
        stop.add(str(num))
        
    # Add these extra stopwords to the list
    # TODO: Look through the corpus and decide which are 
    # extra stopwords needed for this specific domain
    extra = [
        'use', 'using', 'uses', 'used', 'based', 'including', 'include', 'approach','factors','business','risk'
        'wa', 'ha', 'doe', 'item', '1a', 'factor','1b'
            ]
    for word in extra:
        stop.add(word)
    return stop

def cleaning_pipeline(text):
    '''
    Takes a binary string and returns a list of cleaned sentences, stripped of punctuation and lemmatised
    '''

    stopwords = get_stopwords()
    text = normalise_decimals(normalise_acronymns(text))
    text = ' '.join([word for word in text.split() if word not in stopwords])
    sentences = split_into_sentences(text)

    # strip out punctuation and make lowercase
    clean_sentences = []
    for s in sentences:

        # Deal with special cases
        s = re.sub(r'[-/]', ' ', s)

        # Remove all other punctuation
        s = re.sub(r'[^\w\s]', '', s)

        clean_sentences.append(s.lower())

    # pad sentences with delimiters
    text = pad_sentences(clean_sentences)

    # Lemmatise word by word
    lemmas = []
    for word in tokeniser(text):
        lemmas.append(wnl.lemmatize(word))

    return ' '.join(lemmas)

def cleaning_pipeline_df(row):
    row['clean_text'] = cleaning_pipeline(row['text'])
    return row

def df_remove_stopwords(row,column,clean_column):
    stopwords = get_stopwords()
    text = str(row[column])
    row[clean_column] = ' '.join([word for word in text.split() if word not in stopwords])
    return row


# Centralise data into a csv

In [4]:
# Connect to MongoDB
client = MongoClient('127.0.0.1', 27017)
db = client.frtp
collection = db.documents

In [9]:
all_years_df = pd.DataFrame()
for year in tqdm(range(10, 22)):
    result = collection.find({"year": str(year)})
    df = pd.DataFrame(list(result))
    df['clean_text'] = ''
    df['stop_clean_text'] = ''
    df = df.apply(cleaning_pipeline_df,axis=1)
    df = df.apply(lambda row: df_remove_stopwords(row,'clean_text','stop_clean_text'), axis=1)
    df.to_csv(f'data/clean_data/{year}.csv')
    all_years_df = all_years_df.append(df)
all_years_df.to_csv('all_years_df.csv')

  0%|          | 0/12 [00:00<?, ?it/s]

# Build a vocabulary

In [ ]:
vocab = set()
for year in range(15, 21):
    df = pd.read_csv(f'data/clean_csv/{year}.csv')
    
    # The same as above, applied year by year instead.
    t0 = time.time()

    vectorizer = CountVectorizer(strip_accents='ascii', 
                                 ngram_range=(1,ngram_length),
                                 stop_words=get_stopwords(),
                                 min_df=min_yearly_df)


    vector = vectorizer.fit_transform(df.sentences)
    
    # Save the new words
    vocab = vocab.union(vectorizer.vocabulary_.keys())
    
    print(year, len(vocab), time.time()-t0)

    
vocabulary = {}
i = 0
for v in vocab:
    # Remove delimiters
    if start_delimiter in v:
        pass
    elif end_delimiter in v:
        pass
    elif sent_delimiter in v:
        pass
    else:
        vocabulary[v] = i
        i += 1
        
print(len(vocabulary.keys()))

In [ ]:
vectors = []
for year in range(15, 21):
    df = pd.read_csv(f'data/clean_csv/{year}.csv')
    
    # The same as above, applied year by year instead.
    t0 = time.time()


    vectorizer = CountVectorizer(strip_accents='ascii', 
                                 ngram_range=(1,ngram_length),
                                 stop_words=get_stopwords(),
                                 vocabulary=vocabulary)


    vectors.append(vectorizer.fit_transform(df.text))
    
    print(year, time.time()-t0)